In [1]:
### This script looks at all files
### 1) Checks deleted files for md5 related fields and clears them
###                         for qc metric and changes status of qc metric object to deleted
###                         clears the qc metric field
###                         for workflows and deletes all of them
### 2) Checks other files for workflows and deleted old workflows
###                                     and deleted problematic ones (status or rev)
### takes around 20 min

from dcicutils import ff_utils
from functions.notebook_functions import *
#env = 'data'
my_key = get_key('koray_data')

In [9]:
print('started at', datetime.utcnow())

# what kind of files should be searched for worflow run inputs, use url compatible naming

# accepted workflows
# workflow name, accepted revision numbers (0 if none), accetable run time (hours)
workflow_details = [
                    ['md5', ['0'], 12],
                    ['fastqc-0-11-4-1', ['0', '1'], 24],    
                    ['bwa-mem 0.2.5', ['0'],50],
                    ['pairsqc-single 0.2.5', ['0'],12],
                    ['hi-c-processing-bam 0.2.5', ['0'],50],  
                    ['hi-c-processing-pairs 0.2.5', ['0'],100],
                    ['hi-c-processing-pairs-nore 0.2.5', ['0'],100],
                    ['hi-c-processing-pairs-nonorm 0.2.5', ['0'],100],
                    ['hi-c-processing-pairs-nore-nonorm 0.2.5', ['0'],100],
                    ['repliseq-parta 0.2.5', ['0'],100],
                    ['bedGraphToBigWig',['0'], 12]
                   ]
workflow_names = [i[0] for i in workflow_details]

deleted_wfr_no = 0
files_with_deleted_wfr = 0

run_what = 'Custom_no_date'   # Proc or Fastq
date_since = '2018-10-01'
add_deleted = False

file_url = ""
if run_what == 'Proc':
    file_url = '/search/?type=FileProcessed&limit=all&q=date_created%3A%3E%3D' + date_since
    print(file_url)
    files = ff_utils.search_metadata(file_url ,key=my_key)
    # add deleted
    if add_deleted:
        file_del_url = file_url + "&status=deleted"
        files.extend(ff_utils.search_metadata(file_del_url ,key=my_key))
    
elif run_what == 'Fastq':
    file_url = '/search/?type=FileFastq&limit=all&q=date_created%3A%3E%3D' + date_since
    files = ff_utils.search_metadata(file_url, key=my_key)
    # add deleted
    if add_deleted:
        file_del_url = file_url + "&status=deleted"
        files.extend(ff_utils.search_metadata(file_del_url, key=my_key))
    # grab these guys, because they accumulate losts of md5 runs
    for a_test in ['4DNFIO67AFHV','4DNFIXH5OV2H', '4DNFI5RQBUKE']:
        files.insert(0,ff_utils.get_metadata(a_test, key=my_key))

elif run_what == 'Custom':
    file_url = '/search/?type=FileProcessed&limit=all&status=deleted&q=date_created%3A%3E%3D' + date_since
    files = ff_utils.search_metadata(file_url , key=my_key)
    # add deleted
    if add_deleted:
        file_del_url = file_url + "&status=deleted"
        files.extend(ff_utils.search_metadata(file_del_url ,key=my_key))
        
elif run_what == 'Custom_no_date':
    file_url = '/search/?award.project=External&experiments.biosample.biosource.individual.organism.name=chicken&lab.display_title=Job+Dekker%2C+UMMS&type=FileFastq'
    files = ff_utils.search_metadata(file_url, key=my_key)    
    
    
print(len(files), 'files in the system')

started at 2018-11-30 05:26:20.869257
2428 files in the system


In [10]:
print(len(files), 'files in the system')
delete_workflows = input("Do you want to delete old workflowruns (if not, only report will be displayed (y/n))")
counter = 0
del_md5 = 0
del_qc = 0
deleted_output = 0
deleted_wfrs = []
for a_file in files:
    counter += 1  
    if counter % 100 == 0:
        print(counter, files_with_deleted_wfr)
    raw_file = a_file
    deleted_wf = False
    wfr_report = []
    wfrs = raw_file.get('workflow_run_inputs')
    
    # look for md5s
    # to do add more single input runs
    if not wfrs:
        wfrs_url = '/search/?type=WorkflowRun&type=WorkflowRun&workflow.title=md5&input_files.value.accession='+ a_file['accession']
        wfrs = ff_utils.search_metadata(wfrs_url , key=my_key)
#         if len(wfrs)==0:
#             continue
#             #print('file has no wfr', a_file['accession'])
            
    # Delete wfrs if file is deleted
    if raw_file['status'] == 'deleted':
        if delete_workflows.lower() in ['y', 'yes']:
            # clean deleted files of md5 and qc metrics
            for a_field in ['content_md5sum', 'md5sum']:  
                if raw_file.get(a_field):
                    ff_utils.delete_field(raw_file, a_field, key=my_key)
                    del_md5 += 1
            if raw_file.get('quality_metric'):
                qc_uuid = raw_file['quality_metric']['uuid']
                ff_utils.delete_field(raw_file, 'quality_metric', key=my_key)
                # delete quality metrics object
                patch_data = {'status': "deleted"}
                ff_utils.patch_metadata(patch_data, obj_id=qc_uuid , key=my_key)
                del_qc += 1
                
        # delete all workflows for deleted files
        if not wfrs:
            continue
        else:
            wfr_report = get_wfr_report(wfrs, my_key)
            for wfr_to_del in wfr_report:
                if wfr_to_del['status'] != 'deleted':
                    if wfr_to_del['wfr_name'] not in workflow_names:
                        print('Unlisted Workflow', wfr_to_del['wfr_name'], 'deleted file workflow', wfr_to_del['wfr_uuid'], raw_file['accession'])
                    deleted_wf = True
                    deleted_wfr_no += 1
                    
                    ####################################################
                    ## TEMPORARY PIECE##################################
                    if wfr_to_del['status'] == 'released to project':
                        print('saved from deletion', wfr_to_del['wfr_name'], 'deleted file workflow', wfr_to_del['wfr_uuid'], raw_file['accession'])
                        continue
                    if wfr_to_del['status'] == 'released':
                        print('delete released!!!!!', wfr_to_del['wfr_name'], 'deleted file workflow', wfr_to_del['wfr_uuid'], raw_file['accession'])
                        continue  
                    #####################################################
                    
                    print(wfr_to_del['wfr_name'], 'deleted file workflow', wfr_to_del['wfr_uuid'], raw_file['accession'])
                    if delete_workflows.lower() in ['y', 'yes']:
                        patch_data = {'description': "This workflow run is deleted", 'status': "deleted"}
                        deleted_wfrs.append(wfr_to_del['wfr_uuid'])
                        ff_utils.patch_metadata(patch_data, obj_id=wfr_to_del['wfr_uuid'] ,key=my_key)
                        # delete output files of the deleted workflow run
                        if wfr_to_del['outputs']:
                            for out_file in wfr_to_del['outputs']:
                                deleted_output += 1
                                ff_utils.patch_metadata({'status': "deleted"}, obj_id=out_file ,key=my_key)
       
                
    else:
        # get a report on all workflow_runs
        if not wfrs:
            continue
        else:
            wfr_report = get_wfr_report(wfrs, my_key)
            # printTable(wfr_report, ['wfr_name', 'run_time', 'wfr_rev', 'run_time', 'wfr_status'])
            # check if any unlisted wfr in report
            my_wfr_names = [i['wfr_name'] for i in wfr_report]
            unlisted = [x for x in my_wfr_names if x not in workflow_names]
            # report the unlisted ones
            #if unlisted:
                #print('Unlisted Workflow', unlisted, 'skipped in', raw_file['accession'])
                    
            for wf_name,accepted_rev,accepted_run_time in workflow_details:
                #for each type of worklow make a list of old ones, and patch status and description
                sub_wfrs = [i for i in wfr_report if i['wfr_name'] == wf_name]
                if sub_wfrs:
                    active_wfr = sub_wfrs[-1]
                    old_wfrs = sub_wfrs [:-1]
                    # check the status of the most recent workflow
                    if active_wfr['wfr_status'] != 'complete':
                        if active_wfr['wfr_status'] in ['running', 'started'] and active_wfr['run_time'] < accepted_run_time:
                            print(wf_name,'still running for', a_file['accession'])
                        else:
                            old_wfrs.append(active_wfr)
                    elif active_wfr['wfr_rev'] not in accepted_rev:
                        old_wfrs.append(active_wfr)
                    if old_wfrs:
                        for wfr_to_del in old_wfrs:
                            if wfr_to_del['status'] != 'deleted':
                                deleted_wf = True
                                deleted_wfr_no += 1 
                                
                                ####################################################
                                ## TEMPORARY PIECE
                                if wfr_to_del['status'] == 'released to project':
                                    print('saved from deletion',wfr_to_del['wfr_name'], 'old style or dub', wfr_to_del['wfr_uuid'], raw_file['accession'])
                                    continue
                                if wfr_to_del['status'] == 'released':
                                    print('delete released????',wfr_to_del['wfr_name'], 'old style or dub', wfr_to_del['wfr_uuid'], raw_file['accession'])
                                    continue
                                ####################################################

                                print(wfr_to_del['wfr_name'], 'old style or dub', wfr_to_del['wfr_uuid'], raw_file['accession'])
                                
                                if delete_workflows.lower() in ['y', 'yes']:
                                    patch_data = {'description': "This workflow run is deleted", 'status': "deleted"}
                                    deleted_wfrs.append(wfr_to_del['wfr_uuid'])
                                    
                                    ff_utils.patch_metadata(patch_data, obj_id=wfr_to_del['wfr_uuid'] ,key=my_key)
                                    # delete output files of the deleted workflow run
                                    if wfr_to_del['outputs']:
                                        for out_file in wfr_to_del['outputs']:
                                            deleted_output += 1
                                            ff_utils.patch_metadata({'status': "deleted"}, obj_id=out_file ,key=my_key)
    if deleted_wf:
        files_with_deleted_wfr += 1



if delete_workflows.lower() in ['y', 'yes']:
    print(str(deleted_wfr_no),"workflowruns from", str(files_with_deleted_wfr), "files deleted")
else:
    print(str(deleted_wfr_no),"workflowruns from", str(files_with_deleted_wfr), "files need to be deleted")

print(len(deleted_wfrs))
print(del_md5, 'md5 fields deleted')
print(del_qc, 'qc metrics deleted')
print(deleted_output, 'deleted output files')
print('finished at', datetime.utcnow())

2428 files in the system
Do you want to delete old workflowruns (if not, only report will be displayed (y/n))y
100 0
200 0
300 0
400 0


KeyboardInterrupt: 

# Testing trigger and accumulating runs

There are many runs where we are running out of option to utilize the exisiting tool sets
for this cases I would like to implement a new checker, which will be used for testing 
existing frame works and also make sure that newly implemented tools are working in synch with
them.

This are the minor details that we should pay attention to while building complex tools that wrap many components
that are also dynamic. Even a simple version menagement can become problematic when different package management tools 
are used in combination.
